In [3]:
import h5py
import numpy as np
import tifffile
import os
from pathlib import Path

def process_and_convert_h5(h5_file_path):
    """
    Processes a single HDF5 file, converts its 'img' or 'mask' dataset to TIFF,
    and then deletes the original HDF5 file upon success.
    """
    try:
        with h5py.File(h5_file_path, 'r') as h5f:
            # Check if the 'img' dataset exists
            if 'img' in h5f:
                print(f"Processing image file: '{h5_file_path}'...")
                
                # Read the dataset
                image_data = h5f['img'][:]
                
                # Verify that the data is in a suitable format for visualization
                if image_data.ndim == 3 and image_data.shape[-1] >= 4:
                    # Select bands 4, 3, and 2 for a true-color (RGB) composite
                    # h5py and numpy use 0-based indexing
                    rgb_channels = image_data[:, :, [3, 2, 1]]
                    
                    # Generate the output TIFF filename
                    output_tif_path = Path(h5_file_path).with_suffix('.tif')

                    # Save the RGB composite as a TIFF file
                    tifffile.imwrite(output_tif_path, rgb_channels)
                    print(f"Successfully converted '{h5_file_path}' to '{output_tif_path}'")
                    
                    # Delete the original HDF5 file
                    os.remove(h5_file_path)
                    print(f"Successfully removed original file: '{h5_file_path}'")
                else:
                    print(f"Skipping '{h5_file_path}': 'img' dataset is not a suitable 3D array for RGB conversion.")
            
            # Check for mask files
            elif 'mask' in h5f:
                print(f"Processing mask file: '{h5_file_path}'...")
                
                # Read the mask dataset
                mask_data = h5f['mask'][:]

                # Generate the output TIFF filename
                output_tif_path = Path(h5_file_path).with_suffix('.tif')
                
                # Save the mask as a single-channel grayscale TIFF
                # Masks are typically integers, e.g., 0 for background, 1 for landslide
                tifffile.imwrite(output_tif_path, mask_data, photometric='minisblack')
                print(f"Successfully converted '{h5_file_path}' to '{output_tif_path}'")
                
                # Delete the original HDF5 file
                os.remove(h5_file_path)
                print(f"Successfully removed original file: '{h5_file_path}'")
            
            else:
                print(f"Skipping '{h5_file_path}': No 'img' or 'mask' dataset found.")
                
    except FileNotFoundError:
        print(f"Error: The file '{h5_file_path}' was not found.")
    except Exception as e:
        print(f"An unexpected error occurred while processing '{h5_file_path}': {e}")


# --- Main execution block for batch conversion with multiple folders ---
if __name__ == "__main__":
    # Define the root directory to start the search
    dataset_root_directory = '/home/neel/Geog_project/Landslide4Sense'
    
    print(f"Starting conversion for all .h5 files in '{dataset_root_directory}' and its subfolders.")
    
    # Use os.walk to find all files recursively
    for dirpath, dirnames, filenames in os.walk(dataset_root_directory):
        for filename in filenames:
            if filename.endswith('.h5'):
                file_path = os.path.join(dirpath, filename)
                process_and_convert_h5(file_path)

    print("\nBatch conversion process complete.")


Starting conversion for all .h5 files in '/home/neel/Geog_project/Landslide4Sense' and its subfolders.
Processing image file: '/home/neel/Geog_project/Landslide4Sense/ValidData/img/image_180.h5'...
Successfully converted '/home/neel/Geog_project/Landslide4Sense/ValidData/img/image_180.h5' to '/home/neel/Geog_project/Landslide4Sense/ValidData/img/image_180.tif'
Successfully removed original file: '/home/neel/Geog_project/Landslide4Sense/ValidData/img/image_180.h5'
Processing image file: '/home/neel/Geog_project/Landslide4Sense/ValidData/img/image_90.h5'...
Successfully converted '/home/neel/Geog_project/Landslide4Sense/ValidData/img/image_90.h5' to '/home/neel/Geog_project/Landslide4Sense/ValidData/img/image_90.tif'
Successfully removed original file: '/home/neel/Geog_project/Landslide4Sense/ValidData/img/image_90.h5'
Processing image file: '/home/neel/Geog_project/Landslide4Sense/ValidData/img/image_197.h5'...
Successfully converted '/home/neel/Geog_project/Landslide4Sense/ValidData/im